# Pipeline Project

You will be using the provided data to create a machine learning model pipeline.

You must handle the data appropriately in your pipeline to predict whether an
item is recommended by a customer based on their review.
Note the data includes numerical, categorical, and text data.

You should ensure you properly train and evaluate your model.

## The Data

The dataset has been anonymized and cleaned of missing values.

There are 8 features for to use to predict whether a customer recommends or does
not recommend a product.
The `Recommended IND` column gives whether a customer recommends the product
where `1` is recommended and a `0` is not recommended.
This is your model's target/

The features can be summarized as the following:

- **Clothing ID**: Integer Categorical variable that refers to the specific piece being reviewed.
- **Age**: Positive Integer variable of the reviewers age.
- **Title**: String variable for the title of the review.
- **Review Text**: String variable for the review body.
- **Positive Feedback Count**: Positive Integer documenting the number of other customers who found this review positive.
- **Division Name**: Categorical name of the product high level division.
- **Department Name**: Categorical name of the product department name.
- **Class Name**: Categorical name of the product class name.

The target:
- **Recommended IND**: Binary variable stating where the customer recommends the product where 1 is recommended, 0 is not recommended.

## Load Data

In [1]:
import pandas as pd

# Load data
df = pd.read_csv(
    'data/reviews.csv',
)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18442 entries, 0 to 18441
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              18442 non-null  int64 
 1   Age                      18442 non-null  int64 
 2   Title                    18442 non-null  object
 3   Review Text              18442 non-null  object
 4   Positive Feedback Count  18442 non-null  int64 
 5   Division Name            18442 non-null  object
 6   Department Name          18442 non-null  object
 7   Class Name               18442 non-null  object
 8   Recommended IND          18442 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.3+ MB


,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name,Recommended IND
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses,0
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants,1
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses,1
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses,0
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits,1


## Preparing features (`X`) & target (`y`)

In [2]:
data = df

# separate features from labels
X = data.drop('Recommended IND', axis=1)
y = data['Recommended IND'].copy()

print('Labels:', y.unique())
print('Features:')
display(X.head())

Labels: [0 1]
Features:


,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits


In [3]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.1,
    shuffle=True,
    random_state=27,
)

# My Work

## Data Exploration

In [4]:
# Select columns by data type
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns
bool_cols = df.select_dtypes(include=['bool']).columns

print("Numerical columns:", num_cols.tolist())
print("Categorical columns:", cat_cols.tolist())
print("Boolean columns:", bool_cols.tolist())


Numerical columns: ['Clothing ID', 'Age', 'Positive Feedback Count', 'Recommended IND']
Categorical columns: ['Title', 'Review Text', 'Division Name', 'Department Name', 'Class Name']
Boolean columns: []


- Numerical:'Clothing ID', 'Age', 'Positive Feedback Count'
- Categorical:'Division Name', 'Department Name', 'Class Name'
- Text:'Title', 'Review Text'


## Building Pipeline

In [5]:
from sklearn.pipeline import Pipeline

split data into numerical, categorical, and text features

In [6]:
num_features = (
    X
    .select_dtypes(exclude=['object']).columns
    .drop(
          [
               'Age', # More of category than a numerical feature
           ],
    )
)
print('Numerical features:', num_features)

cat_features = (
    X[[
          'Division Name', 
          'Department Name', 
          'Class Name',
          'Age'
       ]].columns
)
print('Categorical features:', cat_features)


text_features = (
    X[[
           'Title', 
           'Review Text'
     ]].columns
)
print ('Review Text features:', text_features)



Numerical features: Index(['Clothing ID', 'Positive Feedback Count'], dtype='object')
Categorical features: Index(['Division Name', 'Department Name', 'Class Name', 'Age'], dtype='object')
Review Text features: Index(['Title', 'Review Text'], dtype='object')


In [7]:
# --- Combine Text Columns ---
#def combine_text_columns(X):
 #   return (X['Title'] + ' ' + X['Review Text']).values

#combine_text_step = FunctionTransformer(combine_text_columns, validate=False)

### Numerical Features Pipeline 

In [8]:
# define pipeline for numerical features called `num_pipeline``
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

num_pipeline = Pipeline([
    (   
         'imputer',
        SimpleImputer(strategy='mean'),
    ),
    (
         'scaler',
        MinMaxScaler(),
    )
])

num_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler())])

### Categorical Features Pipeline

In [9]:
# define pipeline for categorical features called `cat_pipeline`
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    (
          'ordinal_encoder',
          OrdinalEncoder(
               handle_unknown='use_encoded_value',
          unknown_value=-1,
          )
    ),
    (
          'imputer',
          SimpleImputer(
               strategy='most_frequent',
          )
    ),
    (
          'cat_encoder',
          OneHotEncoder(
               sparse_output=False,
               handle_unknown='ignore',
          )
    ),
    
])

cat_pipeline

Pipeline(steps=[('ordinal_encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('cat_encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

### Text Feature Pipeline

In [10]:
import spacy

nlp = spacy.load('en_core_web_sm')


In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OrdinalEncoder, OneHotEncoder
# Create CountCharacter()
# Takes in a string for the character to count
# Outputs the number times that character appears in the text

# --- Combine Text Columns ---
def combine_text_columns(X):
    return (X['Title'] + ' ' + X['Review Text']).values

combine_text_step = FunctionTransformer(combine_text_columns, validate=False)



class CountCharacter(BaseEstimator, TransformerMixin):
    def __init__(self, character: str):
        self.character = character

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [[text.count(self.character)]  for text in X]
    

In [12]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion

character_counts_pipeline = Pipeline([
    (
        'feature_engineering',
        FeatureUnion([
            ('count_spaces', CountCharacter(character=' ')),
            ('count_exclamations', CountCharacter(character='!')),
            ('count_question_marks', CountCharacter(character='?')),
        ]),
    ),
])

# --- Spacy Lemmatizer ---
nlp = spacy.load('en_core_web_sm')

class SpacyLemmatizer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        lemmatized = [
            ' '.join(token.lemma_ for token in doc if not token.is_stop)
            for doc in self.nlp.pipe(X)
        ]
        return lemmatized

# --- TF-IDF Pipeline ---
tfidf_pipeline = Pipeline([
    ('combine_text', combine_text_step),  # <-- FIXED: combine columns first
    ('lemmatizer', SpacyLemmatizer(nlp=nlp)),
    ('tfidf_vectorizer', TfidfVectorizer(stop_words='english')),
])


In [13]:
# TODO: Create your SpacyLemmatizer
class SpacyLemmatizer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp

    # TODO: your code here
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        lemmatized = [
            ' '.join(
            token.lemma_ for token in doc
            if not token.is_stop
            )
            for doc in self.nlp.pipe(X)
        ]
        return lemmatized

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_pipeline = Pipeline([
    (
        'lemmatizer',
        SpacyLemmatizer(nlp=nlp),
    ),
    (
        'tfidf_vectorizer',
        TfidfVectorizer(stop_words='english'),
    ),
])

tfidf_pipeline 

Pipeline(steps=[('lemmatizer',
                 SpacyLemmatizer(nlp=<spacy.lang.en.English object at 0x3007c3110>)),
                ('tfidf_vectorizer', TfidfVectorizer(stop_words='english'))])

In [15]:
from sklearn.pipeline import Pipeline, FeatureUnion

text_pipeline = Pipeline([
    (
        'combine_text',               # Step 1: Title + Review Text
        combine_text_step,
    ),
    (
        'text_features',              # Step 2: parallel text features
        FeatureUnion([
            ('character_counts', character_counts_pipeline),
            ('tfidf', tfidf_pipeline),
        ]),
    ),
])

text_pipeline

Pipeline(steps=[('combine_text',
                 FunctionTransformer(func=<function combine_text_columns at 0x3007aa700>)),
                ('text_features',
                 FeatureUnion(transformer_list=[('character_counts',
                                                 Pipeline(steps=[('feature_engineering',
                                                                  FeatureUnion(transformer_list=[('count_spaces',
                                                                                                  CountCharacter(character=' ')),
                                                                                                 ('count_exclamations',
                                                                                                  CountCharacter(character='!')),
                                                                                                 ('count_question_marks',
                                                                                                  CountCharacter(character='?'))]))])),
                                                ('tfidf',
                                                 Pipeline(steps=[('lemmatizer',
                                                                  SpacyLemmatizer(nlp=<spacy.lang.en.English object at 0x3007c3110>)),
                                                                 ('tfidf_vectorizer',
                                                                  TfidfVectorizer(stop_words='english'))]))]))])

### Combine Feature Engineering Pipelines¶

In [16]:
from sklearn.compose import ColumnTransformer

feature_engineering = ColumnTransformer([
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features),
        ('text', text_pipeline, ['Title', 'Review Text'],   # <<< FIX: only raw text columns
        )
])

feature_engineering

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', MinMaxScaler())]),
                                 Index(['Clothing ID', 'Positive Feedback Count'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('ordinal_encoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1)),
                                                 ('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_encoder'...
                                                                                                   FeatureUnion(transformer_list=[('count_spaces',
                                                                                                                                   CountCharacter(character=' ')),
                                                                                                                                  ('count_exclamations',
                                                                                                                                   CountCharacter(character='!')),
                                                                                                                                  ('count_question_marks',
                                                                                                                                   CountCharacter(character='?'))]))])),
                                                                                 ('tfidf',
                                                                                  Pipeline(steps=[('lemmatizer',
                                                                                                   SpacyLemmatizer(nlp=<spacy.lang.en.English object at 0x3007c3110>)),
                                                                                                  ('tfidf_vectorizer',
                                                                                                   TfidfVectorizer(stop_words='english'))]))]))]),
                                 ['Title', 'Review Text'])])

## Training Pipeline

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

model_pipeline = make_pipeline(
    feature_engineering,
    RandomForestClassifier(random_state=27),
)

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['Clothing ID', 'Positive Feedback Count'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1)),
                                                                  ('imputer',
                                                                   SimpleImputer(stra...
                                                                                                                                                    CountCharacter(character='!')),
                                                                                                                                                   ('count_question_marks',
                                                                                                                                                    CountCharacter(character='?'))]))])),
                                                                                                  ('tfidf',
                                                                                                   Pipeline(steps=[('lemmatizer',
                                                                                                                    SpacyLemmatizer(nlp=<spacy.lang.en.English object at 0x3007c3110>)),
                                                                                                                   ('tfidf_vectorizer',
                                                                                                                    TfidfVectorizer(stop_words='english'))]))]))]),
                                                  ['Title', 'Review Text'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=27))])

### Evaluate Model

In [18]:
from sklearn.metrics import accuracy_score

y_pred_forest_pipeline = model_pipeline.predict(X_test)
accuracy_forest_pipeline = accuracy_score(y_test, y_pred_forest_pipeline)

print('Accuracy:', accuracy_forest_pipeline)

Accuracy: 0.8590785907859079


## Fine-Tuning Pipeline

In [19]:
from sklearn.model_selection import RandomizedSearchCV

# TODO: set parameters to randomly search over
# A couple parameters with 2-5 options each is plenty
my_distributions = dict(
    # TODO: your code here
        randomforestclassifier__max_features=[
               100,
               150,
               250,
        ],
        randomforestclassifier__n_estimators=[
               150,
               200,
        ],
)

In [20]:
param_search = RandomizedSearchCV(
    estimator=model_pipeline,
    param_distributions=my_distributions,
    n_iter=6,     # Try 6 different combinations of parameters
    cv=5,         # Use 5-fold cross-validation
    n_jobs=-1,    # Use all available processors (for multiprocessing)
    refit=True,   # Refit the model using the best parameters found
    verbose=3,    # Output of parameters, score, time
    random_state=27,
)

param_search.fit(X_train, y_train)

# Retrieve the best parameters
param_search.best_params_

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 3/5] END randomforestclassifier__max_features=100, randomforestclassifier__n_estimators=200;, score=0.854 total time= 7.3min
[CV 1/5] END randomforestclassifier__max_features=150, randomforestclassifier__n_estimators=200;, score=0.868 total time=71.1min
[CV 4/5] END randomforestclassifier__max_features=250, randomforestclassifier__n_estimators=150;, score=0.867 total time=23.1min
[CV 2/5] END randomforestclassifier__max_features=100, randomforestclassifier__n_estimators=150;, score=0.861 total time= 7.3min
[CV 3/5] END randomforestclassifier__max_features=150, randomforestclassifier__n_estimators=150;, score=0.864 total time=71.1min
[CV 3/5] END randomforestclassifier__max_features=250, randomforestclassifier__n_estimators=150;, score=0.874 total time=23.2min
[CV 2/5] END randomforestclassifier__max_features=100, randomforestclassifier__n_estimators=200;, score=0.860 total time= 7.2min
[CV 5/5] END randomforestclassifier__

{'randomforestclassifier__n_estimators': 200,
 'randomforestclassifier__max_features': 250}

In [21]:
model_best = param_search.best_estimator_
model_best

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['Clothing ID', 'Positive Feedback Count'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1)),
                                                                  ('imputer',
                                                                   SimpleImputer(stra...
                                                                                                                                                   ('count_question_marks',
                                                                                                                                                    CountCharacter(character='?'))]))])),
                                                                                                  ('tfidf',
                                                                                                   Pipeline(steps=[('lemmatizer',
                                                                                                                    SpacyLemmatizer(nlp=<spacy.lang.en.English object at 0x32f8ab5d0>)),
                                                                                                                   ('tfidf_vectorizer',
                                                                                                                    TfidfVectorizer(stop_words='english'))]))]))]),
                                                  ['Title', 'Review Text'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_features=250, n_estimators=200,
                                        random_state=27))])

In [22]:
y_pred_forest_pipeline = model_best.predict(X_test)
accuracy_forest_pipeline = accuracy_score(y_test, y_pred_forest_pipeline)

print('Accuracy:', accuracy_forest_pipeline)

[CV 4/5] END randomforestclassifier__max_features=100, randomforestclassifier__n_estimators=150;, score=0.855 total time= 7.1min
[CV 4/5] END randomforestclassifier__max_features=100, randomforestclassifier__n_estimators=200;, score=0.855 total time=71.1min
[CV 2/5] END randomforestclassifier__max_features=150, randomforestclassifier__n_estimators=200;, score=0.867 total time=23.1min
[CV 5/5] END randomforestclassifier__max_features=250, randomforestclassifier__n_estimators=150;, score=0.870 total time=51.1min
Accuracy: 0.8731707317073171
[CV 1/5] END randomforestclassifier__max_features=100, randomforestclassifier__n_estimators=150;, score=0.864 total time= 7.2min
[CV 5/5] END randomforestclassifier__max_features=100, randomforestclassifier__n_estimators=200;, score=0.858 total time=71.1min
[CV 3/5] END randomforestclassifier__max_features=150, randomforestclassifier__n_estimators=200;, score=0.864 total time=23.1min
[CV 1/5] END randomforestclassifier__max_features=250, randomforestc